#Интерполяция

Задача:
1. Восстановить пропуски в данных с метеостанции с помощью линейной и квадратичной интерполяции.
2. Реализовать алгоритмы линейной и квадратичной интерполяции.

In [32]:
import pandas as pd
import numpy as np

###Линейная интерполяция

In [33]:
def is_null(data, row, col):
  return pd.isnull(data.iloc[row, col])

In [34]:
def find_passes(data, row, col):
  left_col = col - 1
  right_col = col + 1

  # сдвиг влево
  while left_col >= 0 and is_null(data, row, left_col):
      left_col -= 1

  # сдвиг вправо
  while right_col < data.shape[1] and is_null(data, row, right_col):
      right_col += 1

  return left_col, right_col

In [35]:
def first_cell_skipping_solution(data, row, left_col):
  left_col += 1

  while left_col < data.shape[1] and is_null(data, row, left_col):
    # поиск последнего пропуска в окне (для возможности интерполяции)
    left_col += 1
    right_col = left_col + 1

    if not is_null(data, row, left_col) and not is_null(data, row, right_col):
      break

  return left_col, right_col

In [36]:
def last_cell_skipping_solution(data, row, right_col):
  right_col -= 1

  while right_col > 0 and is_null(data, row, right_col):
    # поиск первого пропуска в окне (для возможности интерполяции)
    right_col -= 1
    left_col = right_col - 1

    if not is_null(data, row, left_col) and not is_null(data, row, right_col):
      break

  return left_col, right_col

In [37]:
def linear_interpolation(data, row, col):

    left_col, right_col = find_passes(data, row, col)

    if left_col == -1:
      left_col, right_col = first_cell_skipping_solution(data, row, left_col)

    if right_col == data.shape[1]:
      left_col, right_col = last_cell_skipping_solution(data, row, right_col)

    if left_col >= 0 and right_col < data.shape[1]:
      #print(left_col,' ', right_col, '\n')
      value = data.iloc[row, left_col] + (float(data.iloc[row, right_col]) - float(data.iloc[row, left_col])) * (col - left_col) / (right_col - left_col)

    return round(value, 2)

###Квадратичная интерполяция

In [38]:
def find_passes_squared(data, row, col):
  left_left_col = col - 2
  left_col = col - 1
  right_col = col + 1

  while left_left_col > 1 and is_null(data, row, left_left_col):
    left_left_col = left_col - 1

  while left_col > 0 and is_null(data, row, left_col):
    left_col -= 1

  while right_col < data.shape[1] and is_null(data, row, right_col):
    right_col += 1

  return left_left_col, left_col, right_col

In [39]:
def first_cell_skipping_solution_squared(data, row, left_left_col):
  left_left_col += 2
  left_col = left_left_col + 1
  right_col = left_col + 1

  # поиск последнего пропуска в окне (для возможности интерполяции)
  while left_left_col < data.shape[1] - 2 and is_null(data, row, left_left_col):
    left_left_col += 1
    left_col = left_left_col + 1
    right_col = left_col + 1

  while left_col < data.shape[1] - 1 and is_null(data, row, left_col):
    left_col = left_left_col + 1
    right_col = left_col + 1

  while right_col < data.shape[1] and is_null(data, row, right_col):
    right_col = left_col + 1

  return left_left_col, left_col, right_col

In [40]:
def second_cell_skipping_solution_squared(data, row, left_left_col):
  left_left_col += 1
  left_col = left_left_col + 1
  right_col = left_col + 1

  # поиск последнего пропуска в окне (для возможности интерполяции)
  while left_left_col < data.shape[1] - 2 and is_null(data, row, left_left_col):
    left_left_col += 1
    left_col = left_left_col + 1
    right_col = left_col + 1

  while left_col < data.shape[1] - 1 and is_null(data, row, left_col):
    left_col += 1
    right_col = left_col + 1

  while right_col < data.shape[1] and is_null(data, row, right_col):
    right_col = left_col + 1

  return left_left_col, left_col, right_col

In [41]:
def last_cell_skipping_solution_squared(data, row, right_col):
  right_col -= 1
  left_col = right_col - 1
  left_left_col = left_col - 1

  while right_col > 2 and is_null(data, row, right_col):
    right_col -= 1
    left_col = right_col - 1
    left_left_col = left_col - 1

  while left_col > 1 and is_null(data, row, left_col):
    left_col = right_col - 1
    left_left_col = left_col - 1

  while left_left_col > 0 and is_null(data, row, left_left_col):
    left_left_col = left_col - 1

  return left_left_col, left_col, right_col

In [42]:
def quadratic_interpolation(data, row, col):

    left_left_col, left_col, right_col = find_passes_squared(data, row, col)

    if left_left_col == -2:
        left_left_col, left_col, right_col = first_cell_skipping_solution_squared(data, row, left_left_col)

    if left_left_col == -1:
        left_left_col, left_col, right_col = second_cell_skipping_solution_squared(data, row, left_left_col)

    if right_col == data.shape[1]:
        left_left_col, left_col, right_col = last_cell_skipping_solution_squared(data, row, right_col)

    if left_left_col >= 0 and right_col < data.shape[1]:
        x = col
        x1 = left_left_col
        x2 = left_col
        x3 = right_col
        y1 = data.iloc[row, left_left_col]
        y2 = data.iloc[row, left_col]
        y3 = data.iloc[row, right_col]

        out = y1*((x-x2)*(x-x3))/((x1-x2)*(x1-x3)) + y2*((x-x1)*(x-x3))/((x2-x1)*(x2-x3)) + y3*((x-x1)*(x-x2))/((x3-x1)*(x3-x2))

        return round(out, 2)

###Заполнение пропусков

In [46]:
def apply_linear_interpolation(data):
  for row in range(data.shape[0]):
    for col in range(data.shape[1]):
        if is_null(data, row, col):
            data.iloc[row, col] = linear_interpolation(data, row, col)

  return data

In [47]:
def apply_quadratic_interpolation(data):
  for row in range(data.shape[0]):
    for col in range(data.shape[1]):
        if is_null(data, row, col):
            data.iloc[row, col] = quadratic_interpolation(data, row, col)

  return data

In [48]:
temperature_data = pd.read_csv('temperature.csv', header=2, usecols=list(range(0, 24)), index_col=0, decimal=",")
temperature_data_full = temperature_data.apply(pd.to_numeric, errors='coerce')

temperature_data_full_linear_interpolation = apply_linear_interpolation(temperature_data_full)
temperature_data_full_linear_interpolation.to_csv('temperature_data_full_linear_interpolation.csv')
print("Файл с линейной интерполяцией температуры - temperature_data_full_linear_interpolation.csv")

temperature_data_full_quadratic_interpolation = apply_quadratic_interpolation(temperature_data_full)
temperature_data_full_quadratic_interpolation.to_csv('temperature_data_full_quadratic_interpolation.csv')
print("Файл с квадратичной интерполяцией температуры - temperature_data_full_quadratic_interpolation.csv")

Файл с линейной интерполяцией температуры - temperature_data_full_linear_interpolation.csv
Файл с квадратичной интерполяцией температуры - temperature_data_full_quadratic_interpolation.csv


In [51]:
dew_point_data = pd.read_csv('dew_point.csv', header=2, usecols=list(range(0, 24)), index_col=0, decimal=",")
dew_point_data_full = dew_point_data.apply(pd.to_numeric, errors='coerce')
dew_point_data_full = dew_point_data_full.iloc[0:31]

dew_point_data_full_linear_interpolation = apply_linear_interpolation(dew_point_data_full)
dew_point_data_full_linear_interpolation.to_csv('dew_point_data_full_linear_interpolation.csv')
print("Файл с линейной интерполяцией температуры - dew_point_data_full_linear_interpolation.csv")

dew_point_data_full_quadratic_interpolation = apply_quadratic_interpolation(dew_point_data_full)
dew_point_data_full_quadratic_interpolation.to_csv('dew_point_data_full_quadratic_interpolation.csv')
print("Файл с квадратичной интерполяцией температуры - dew_point_data_full_quadratic_interpolation.csv")

Файл с линейной интерполяцией температуры - dew_point_data_full_linear_interpolation.csv
Файл с квадратичной интерполяцией температуры - dew_point_data_full_quadratic_interpolation.csv


In [52]:
atmospheric_pressure_data = pd.read_csv('atmospheric_pressure.csv', header=2, usecols=list(range(0, 24)), index_col=0, decimal=",")
atmospheric_pressure_data_full = atmospheric_pressure_data.apply(pd.to_numeric, errors='coerce')

atmospheric_pressure_data_full_linear_interpolation = apply_linear_interpolation(atmospheric_pressure_data_full)
atmospheric_pressure_data_full_linear_interpolation.to_csv('atmospheric_pressure_data_full_linear_interpolation.csv')
print("Файл с линейной интерполяцией температуры - atmospheric_pressure_data_full_linear_interpolation.csv")

atmospheric_pressure_data_full_quadratic_interpolation = apply_quadratic_interpolation(atmospheric_pressure_data)
atmospheric_pressure_data_full_quadratic_interpolation.to_csv('atmospheric_pressure_data_full_quadratic_interpolation.csv')
print("Файл с квадратичной интерполяцией температуры - atmospheric_pressure_data_full_quadratic_interpolation.csv")

Файл с линейной интерполяцией температуры - atmospheric_pressure_data_full_linear_interpolation.csv
Файл с квадратичной интерполяцией температуры - atmospheric_pressure_data_full_quadratic_interpolation.csv
